In [1]:
from __future__ import print_function

from datetime import datetime
import time
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras
from keras.datasets import mnist
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv1D, MaxPooling1D, Conv2D, MaxPooling2D
from keras import backend as K
import numpy as np
import os
import random
from random import getrandbits
from scipy.io import wavfile
from subprocess import check_output
from keras.models import model_from_json
from scipy import signal
import librosa

def timer():
    time_start = [int(time.time())]
    time_start_overall = [int(time.time())]

    def time_taken():
        time_now = int(time.time())
        om, os = divmod(time_now - time_start_overall[0], 60)
        m, s = divmod(time_now - time_start[0], 60)
        time_start[0] = time_now
        return ' overall_time: ' + str(om) + 'm' + str(os) + 's' + ' time_from_previous_call: ' + str(m) + 'm' + str(s) + 's'
    return time_taken
time_taken = timer()

Using TensorFlow backend.


In [2]:
model = load_model('model-pseudo-88-melspec-inception-resnet-v2-1-08-0.99-0.04.h5')

In [3]:
unknowns = 'bed bird cat dog eight five four happy house marvin nine one seven sheila six three tree two wow zero'.split()
knowns = 'yes no up down left right on off stop go'.split()
silence = 'silence'.split()
labels = knowns + silence + ['unknown']

In [4]:
print('fname,label,' + ','.join(labels))

fname,label,yes,no,up,down,left,right,on,off,stop,go,silence,unknown


In [5]:
def get_melspectrogram(samples):
    S = librosa.feature.melspectrogram(samples, sr=sr, n_mels=72, hop_length=223, n_fft=512)
    #S = librosa.feature.melspectrogram(samples, sr=sr, n_mels=32)
    spec = librosa.power_to_db(S, ref=np.max)
    spec = np.repeat(spec[np.newaxis,:,:,np.newaxis], 3, axis=3)
    return spec
# 150 * 150
def get_mel_of_150_150(samples):
    S = librosa.feature.melspectrogram(samples, sr=sr, n_mels=150, hop_length=107, n_fft=512)
    spec = librosa.power_to_db(S, ref=np.max)
    spec = np.repeat(spec[np.newaxis,:,:,np.newaxis], 3, axis=3)
    return spec

In [6]:
test_dir = '../test/audio/'
file_names = []
for filename in os.listdir(test_dir):
    file_names.append(filename)
total_files = len(file_names)
print('total files: ', total_files)
num_files_read = 0
idx = 0
nprint = 5000
sr = 16000
batch_size = 64
X = np.random.randn(nprint, 150, 150, 3)
f1 = open('model-pseudo-88-melspec-inception-resnet-v2-1-08-0.99-0.04.h5-prediction.csv', 'w')
f2 = open('model-pseudo-88-melspec-inception-resnet-v2-1-08-0.99-0.04.h5-prediction-detailed.csv', 'w')
f1.write('fname,label\n')
files = []
for filename in file_names:
    files.append(filename)
    samples, _ = librosa.load(test_dir + filename, sr=sr)
    spec = get_mel_of_150_150(samples)
    X[idx, :, :] = spec
    num_files_read += 1
    idx += 1
    if num_files_read % nprint == 0:
        print('num_files_read: ', num_files_read, time_taken())
        pred = model.predict(X, batch_size=batch_size, verbose=0)
        predictions = np.argmax(pred, axis=1)
        for i in range(len(predictions)):
            label = labels[predictions[i]]
            f1.write('{},{}\n'.format(files[i], label))
            f2.write(files[i] + ',' + label + ',' + str(pred[i][predictions[i]]) + ',' + ','.join([str(x) for x in pred[i]]) + '\n')
        files = []
        if (total_files - num_files_read < nprint):
            X = np.random.randn(total_files - num_files_read, 150, 150, 3)
        idx = 0
if len(files):
    pred = model.predict(X, batch_size=batch_size, verbose=0)
    predictions = np.argmax(pred, axis=1)
    for i in range(len(predictions)):
        label = labels[predictions[i]]
        f1.write('{},{}\n'.format(files[i], label))
        f2.write(files[i] + ',' + label + ',' + str(pred[i][predictions[i]]) + ',' + ','.join([str(x) for x in pred[i]]) + '\n')
f1.close()
f2.close()
print('done', time_taken())

total files:  158538
num_files_read:  5000  overall_time: 1m50s time_from_previous_call: 1m50s
num_files_read:  10000  overall_time: 2m43s time_from_previous_call: 0m53s
num_files_read:  15000  overall_time: 3m34s time_from_previous_call: 0m51s
num_files_read:  20000  overall_time: 4m25s time_from_previous_call: 0m51s
num_files_read:  25000  overall_time: 5m15s time_from_previous_call: 0m50s
num_files_read:  30000  overall_time: 6m6s time_from_previous_call: 0m51s
num_files_read:  35000  overall_time: 6m56s time_from_previous_call: 0m50s
num_files_read:  40000  overall_time: 7m48s time_from_previous_call: 0m52s
num_files_read:  45000  overall_time: 8m38s time_from_previous_call: 0m50s
num_files_read:  50000  overall_time: 9m29s time_from_previous_call: 0m51s
num_files_read:  55000  overall_time: 10m21s time_from_previous_call: 0m52s
num_files_read:  60000  overall_time: 11m11s time_from_previous_call: 0m50s
num_files_read:  65000  overall_time: 12m1s time_from_previous_call: 0m50s
num_